# EPR Lineshape Functions

This notebook demonstrates the EPR lineshape functions available in the `epyr.lineshapes` module.

We will show Gaussian, Lorentzian, and Voigt profiles with their derivatives, and demonstrate fitting noisy synthetic data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import sys

# Add epyr to path
sys.path.append('../../')

from epyr.lineshapes import gaussian, lorentzian, voigtian, Lineshape, lshape, pseudo_voigt

# Create magnetic field range
B = np.linspace(-20, 20, 1000)  # mT

print("EPyR Lineshapes module loaded successfully")

## Basic Lineshape Functions

Generate the three main EPR lineshape functions: Gaussian, Lorentzian, and Voigt.

In [ ]:
# Parameters for lineshapes
center = 0.0  # mT
width = 4.0   # mT

# Generate basic lineshapes
gauss = gaussian(B, center=center, width=width)
lorentz = lorentzian(B, center=center, width=width)
voigt = voigtian(B, center=center, widths=(width/2, width/2))  # (gaussian_width, lorentzian_width)

# Plot comparison
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(B, gauss, 'b-', label='Gaussian', linewidth=2)
plt.plot(B, lorentz, 'r-', label='Lorentzian', linewidth=2)
plt.plot(B, voigt, 'g-', label='Voigt', linewidth=2)
plt.xlabel('Magnetic Field (mT)')
plt.ylabel('Signal Intensity')
plt.title('EPR Lineshape Functions')
plt.legend()
plt.grid(True, alpha=0.3)

# Zoom in to show differences
plt.subplot(1, 2, 2)
plt.plot(B, gauss, 'b-', label='Gaussian', linewidth=2)
plt.plot(B, lorentz, 'r-', label='Lorentzian', linewidth=2)
plt.plot(B, voigt, 'g-', label='Voigt', linewidth=2)
plt.xlabel('Magnetic Field (mT)')
plt.ylabel('Signal Intensity')
plt.title('Lineshape Comparison (Zoomed)')
plt.xlim(-10, 10)
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Derivative Lineshapes

EPR spectra are often recorded as first derivatives. The lineshape functions support derivative calculation.

In [ ]:
# Generate derivative lineshapes
gauss_1st = gaussian(B, center=center, width=width, derivative=1)
gauss_2nd = gaussian(B, center=center, width=width, derivative=2)
lorentz_1st = lorentzian(B, center=center, width=width, derivative=1)

# Plot derivatives
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Absorption vs 1st derivative - Gaussian
axes[0, 0].plot(B, gauss, 'b-', label='Absorption', linewidth=2)
axes[0, 0].plot(B, gauss_1st, 'r-', label='1st Derivative', linewidth=2)
axes[0, 0].set_title('Gaussian: Absorption vs 1st Derivative')
axes[0, 0].set_xlabel('Magnetic Field (mT)')
axes[0, 0].set_ylabel('Signal')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Absorption vs 1st derivative - Lorentzian
axes[0, 1].plot(B, lorentz, 'b-', label='Absorption', linewidth=2)
axes[0, 1].plot(B, lorentz_1st, 'r-', label='1st Derivative', linewidth=2)
axes[0, 1].set_title('Lorentzian: Absorption vs 1st Derivative')
axes[0, 1].set_xlabel('Magnetic Field (mT)')
axes[0, 1].set_ylabel('Signal')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Compare 1st derivatives
axes[1, 0].plot(B, gauss_1st, 'b-', label='Gaussian 1st', linewidth=2)
axes[1, 0].plot(B, lorentz_1st, 'r-', label='Lorentzian 1st', linewidth=2)
axes[1, 0].set_title('1st Derivative Comparison')
axes[1, 0].set_xlabel('Magnetic Field (mT)')
axes[1, 0].set_ylabel('Signal')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 2nd derivative
axes[1, 1].plot(B, gauss_1st, 'g-', label='1st Derivative', linewidth=2)
axes[1, 1].plot(B, gauss_2nd, 'orange', label='2nd Derivative', linewidth=2)
axes[1, 1].set_title('Gaussian: 1st vs 2nd Derivative')
axes[1, 1].set_xlabel('Magnetic Field (mT)')
axes[1, 1].set_ylabel('Signal')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Unified Lineshape Class

The Lineshape class provides a unified interface for all lineshape functions with consistent parameters.

In [ ]:
# Using the unified Lineshape class
gauss_shape = Lineshape('gaussian', width=4.0)
lorentz_shape = Lineshape('lorentzian', width=4.0)
voigt_shape = Lineshape('voigt', width=(2.0, 2.0))  # (gaussian_width, lorentzian_width)
pseudo_voigt_shape = Lineshape('pseudo_voigt', width=4.0, alpha=0.5)

# Generate lineshapes using the class
gauss_class = gauss_shape(B, center=0)
lorentz_class = lorentz_shape(B, center=0)
voigt_class = voigt_shape(B, center=0)
pseudo_voigt_class = pseudo_voigt_shape(B, center=0)

# Plot unified interface results
plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
plt.plot(B, gauss_class, 'b-', label='Gaussian', linewidth=2)
plt.plot(B, lorentz_class, 'r-', label='Lorentzian', linewidth=2)
plt.plot(B, voigt_class, 'g-', label='Voigt', linewidth=2)
plt.plot(B, pseudo_voigt_class, 'm-', label='Pseudo-Voigt (50/50)', linewidth=2)
plt.xlabel('Magnetic Field (mT)')
plt.ylabel('Signal Intensity')
plt.title('Lineshape Class Interface')
plt.legend()
plt.grid(True, alpha=0.3)

# Show different pseudo-Voigt mixing ratios using direct function
plt.subplot(2, 1, 2)
alphas = [0.2, 0.5, 0.8]
for alpha in alphas:
    pv_signal = lshape(B, center=0, width=4.0, alpha=alpha)
    plt.plot(B, pv_signal, label=f'Pseudo-Voigt α={alpha}', linewidth=2)

plt.xlabel('Magnetic Field (mT)')
plt.ylabel('Signal Intensity')
plt.title('Pseudo-Voigt with Different Mixing Parameters')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Alternative Pseudo-Voigt Function

The module also provides a direct pseudo_voigt function with eta parameter.

In [ ]:
# Using pseudo_voigt function directly
plt.figure(figsize=(10, 6))

eta_values = [0.0, 0.3, 0.5, 0.7, 1.0]
for eta in eta_values:
    pv_signal = pseudo_voigt(B, center=0, width=4.0, eta=eta)
    label = f'η={eta} ('
    if eta == 0.0:
        label += 'Pure Lorentzian)'
    elif eta == 1.0:
        label += 'Pure Gaussian)'
    else:
        label += 'Mixed)'
    plt.plot(B, pv_signal, label=label, linewidth=2)

plt.xlabel('Magnetic Field (mT)')
plt.ylabel('Signal Intensity')
plt.title('Pseudo-Voigt Function with Different η Parameters')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("Note: η=0 gives pure Lorentzian, η=1 gives pure Gaussian")

## Multi-component Spectra

Create synthetic EPR spectra with multiple components, as often seen in real EPR measurements.

In [ ]:
# Create multi-component spectrum
# Note: These functions are area-normalized, so we scale by multiplying
component1 = 1.0 * gaussian(B, center=-5, width=2.5)
component2 = 0.7 * lorentzian(B, center=3, width=3.0)
component3 = 0.5 * gaussian(B, center=8, width=1.8)

# Total spectrum
total_spectrum = component1 + component2 + component3

# Plot multi-component spectrum
plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
plt.plot(B, component1, 'b--', label='Component 1 (Gaussian)', alpha=0.7)
plt.plot(B, component2, 'r--', label='Component 2 (Lorentzian)', alpha=0.7)
plt.plot(B, component3, 'g--', label='Component 3 (Gaussian)', alpha=0.7)
plt.plot(B, total_spectrum, 'k-', label='Total Spectrum', linewidth=2)
plt.xlabel('Magnetic Field (mT)')
plt.ylabel('Signal Intensity')
plt.title('Multi-component EPR Spectrum')
plt.legend()
plt.grid(True, alpha=0.3)

# First derivative of multi-component spectrum
deriv1 = 1.0 * gaussian(B, center=-5, width=2.5, derivative=1)
deriv2 = 0.7 * lorentzian(B, center=3, width=3.0, derivative=1)
deriv3 = 0.5 * gaussian(B, center=8, width=1.8, derivative=1)
total_deriv = deriv1 + deriv2 + deriv3

plt.subplot(2, 1, 2)
plt.plot(B, deriv1, 'b--', alpha=0.7)
plt.plot(B, deriv2, 'r--', alpha=0.7)
plt.plot(B, deriv3, 'g--', alpha=0.7)
plt.plot(B, total_deriv, 'k-', linewidth=2, label='Total 1st Derivative')
plt.xlabel('Magnetic Field (mT)')
plt.ylabel('Signal Intensity')
plt.title('First Derivative Multi-component Spectrum')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Fitting Noisy EPR Data

Demonstrate fitting capabilities by adding noise to synthetic data and fitting with lineshape functions.

In [ ]:
# Generate synthetic noisy EPR data
np.random.seed(42)  # For reproducible results

# True parameters
true_center = 2.0
true_width = 3.5
true_amplitude = 0.8

# Generate clean signal (first derivative)
clean_signal = true_amplitude * gaussian(B, center=true_center, width=true_width, derivative=1)

# Add noise
noise_level = 0.05
noise = np.random.normal(0, noise_level, len(B))
noisy_signal = clean_signal + noise

# Define fitting function
def gauss_fit_func(x, center, width, amplitude):
    return amplitude * gaussian(x, center=center, width=width, derivative=1)

# Initial guess
initial_guess = [0.0, 4.0, 1.0]

# Perform fit
popt, pcov = curve_fit(gauss_fit_func, B, noisy_signal, p0=initial_guess)
fitted_signal = gauss_fit_func(B, *popt)

# Calculate fit quality
residuals = noisy_signal - fitted_signal
r_squared = 1 - np.sum(residuals**2) / np.sum((noisy_signal - np.mean(noisy_signal))**2)

print(f"Fit Results:")
print(f"True parameters: center={true_center}, width={true_width}, amplitude={true_amplitude}")
print(f"Fitted parameters: center={popt[0]:.2f}, width={popt[1]:.2f}, amplitude={popt[2]:.2f}")
print(f"R-squared: {r_squared:.4f}")

In [ ]:
# Plot fitting results
plt.figure(figsize=(14, 10))

# Main fit plot
plt.subplot(2, 2, 1)
plt.plot(B, clean_signal, 'g-', label='True Signal', linewidth=2, alpha=0.8)
plt.plot(B, noisy_signal, 'b.', label='Noisy Data', alpha=0.6, markersize=1)
plt.plot(B, fitted_signal, 'r-', label='Fitted Signal', linewidth=2)
plt.xlabel('Magnetic Field (mT)')
plt.ylabel('Signal Intensity')
plt.title(f'Gaussian Fit (R² = {r_squared:.4f})')
plt.legend()
plt.grid(True, alpha=0.3)

# Residuals
plt.subplot(2, 2, 2)
plt.plot(B, residuals, 'k-', linewidth=1)
plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
plt.xlabel('Magnetic Field (mT)')
plt.ylabel('Residuals')
plt.title('Fit Residuals')
plt.grid(True, alpha=0.3)

# Compare different lineshapes for fitting
def lorentz_fit_func(x, center, width, amplitude):
    return amplitude * lorentzian(x, center=center, width=width, derivative=1)

# Fit with Lorentzian
popt_lorentz, _ = curve_fit(lorentz_fit_func, B, noisy_signal, p0=initial_guess)
fitted_lorentz = lorentz_fit_func(B, *popt_lorentz)
r_squared_lorentz = 1 - np.sum((noisy_signal - fitted_lorentz)**2) / np.sum((noisy_signal - np.mean(noisy_signal))**2)

plt.subplot(2, 2, 3)
plt.plot(B, noisy_signal, 'b.', label='Noisy Data', alpha=0.6, markersize=1)
plt.plot(B, fitted_signal, 'r-', label=f'Gaussian (R²={r_squared:.3f})', linewidth=2)
plt.plot(B, fitted_lorentz, 'orange', label=f'Lorentzian (R²={r_squared_lorentz:.3f})', linewidth=2)
plt.xlabel('Magnetic Field (mT)')
plt.ylabel('Signal Intensity')
plt.title('Lineshape Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

# Parameter uncertainty
param_errors = np.sqrt(np.diag(pcov))
plt.subplot(2, 2, 4)
params = ['Center', 'Width', 'Amplitude']
fitted_values = popt
errors = param_errors
plt.errorbar(params, fitted_values, yerr=errors, fmt='ro', capsize=5, capthick=2)
plt.ylabel('Parameter Value')
plt.title('Fitted Parameters with Uncertainties')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nParameter uncertainties:")
for i, param in enumerate(['center', 'width', 'amplitude']):
    print(f"{param}: {popt[i]:.3f} ± {param_errors[i]:.3f}")

## Summary

The EPyR lineshapes module provides:

**Core Functions:**
- `gaussian(x, center, width, derivative=0)` - Gaussian lineshapes with derivatives
- `lorentzian(x, center, width, derivative=0)` - Lorentzian lineshapes with derivatives  
- `voigtian(x, center, widths, derivative=0)` - True Voigt profiles (widths as tuple)
- `pseudo_voigt(x, center, width, eta=0.5)` - Pseudo-Voigt with mixing parameter
- `lshape(x, center, width, alpha=1.0)` - General lineshape function

**Unified Interface:**
- `Lineshape` class for consistent parameter handling
- Support for different lineshape types through single interface

**Key Features:**
- Derivative calculation (0th, 1st, 2nd order)
- Multi-component spectrum modeling
- Compatible with scipy.optimize for fitting
- Area-normalized functions for quantitative analysis
- Optimized for EPR spectroscopy applications

**Important Notes:**
- Functions are area-normalized, so scale by multiplication for amplitude
- Voigt function requires widths as tuple: (gaussian_width, lorentzian_width)
- Pseudo-Voigt has two variants: eta parameter (0=Lorentz, 1=Gauss) or alpha parameter (1=Gauss, 0=Lorentz)